## VERSION 6: now we are breaking into little part and putting it together to work

In [ ]:
# this is the first portion of the code multi agent system and it works.

from openai import OpenAI
import time
from datetime import datetime

# Setup DeepSeek client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

def generate_followup_question(learning_goal):
    prompt = f"""You are a motivational learning coach. A learner just said they want to learn: "{learning_goal}".

Generate one open-ended, thoughtful question that will help them expand deeply on their personal motivation, challenges, or relationship to this topic. Be emotionally engaging and not too abstract."""

    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1-0528-qwen3-8b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300
        )
        print("✅ DeepSeek response received.")
        print("🔧 FULL RESPONSE:", response)

        content = response.choices[0].message.content.strip() if response.choices else ""
        if not content:
            print("⚠️ Empty content. Using fallback.")
            content = "What does writing mean to you personally, and how do you want it to impact your future?"

        return content

    except Exception as e:
        print(f"⚠️ DeepSeek generation failed: {e}")
        return "What inspires your desire to write, and what do you hope to express through it?"


def agent_one():
    print("🎯 Welcome to the Adaptive Learning System (Prototype Agent 1)")
    learning_goal = input("🧠 What are you trying to learn today?\n> ")

    # Generate personalized follow-up
    followup_question = generate_followup_question(learning_goal)
    print(f"\n🔍 Reflect on this:\n{followup_question}\n")

    # Time response (to evaluate latency later)
    start_time = time.time()
    user_response = input("✍️ Your response:\n> ")
    end_time = time.time()

    time_taken = round(end_time - start_time, 2)
    timestamp = datetime.now().isoformat()

    print("\n🕒 Time to respond:", time_taken, "seconds")
    print("🗓️ Timestamp:", timestamp)

    return {
        "learning_goal": learning_goal,
        "followup_question": followup_question,
        "response": user_response,
        "time_taken": time_taken,
        "timestamp": timestamp
    }

# Run prototype
if __name__ == "__main__":
    result = agent_one()
    print("\n✅ DATA CAPTURED:")
    for k, v in result.items():
        print(f"{k}: {v}")


In [ ]:
from openai import OpenAI
import time
from datetime import datetime

# Setup DeepSeek client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

def generate_followup_question(learning_goal):
    prompt = f"""You are a motivational learning coach. A learner just said they want to learn: "{learning_goal}".

Generate one open-ended, thoughtful question that will help them expand deeply on their personal motivation, challenges, or relationship to this topic. Be emotionally engaging and not too abstract."""

    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1-0528-qwen3-8b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300
        )
        print("✅ DeepSeek response received.")
        print("🔧 FULL RESPONSE:", response)

        content = response.choices[0].message.content.strip() if response.choices else ""
        if not content:
            print("⚠️ Empty content. Using fallback.")
            content = "What does writing mean to you personally, and how do you want it to impact your future?"

        return content

    except Exception as e:
        print(f"⚠️ DeepSeek generation failed: {e}")
        return "What inspires your desire to write, and what do you hope to express through it?"


def agent_one():
    print("🎯 Welcome to the Adaptive Learning System (Prototype Agent 1)")
    learning_goal = input("🧠 What are you trying to learn today?\n> ")

    # Generate personalized follow-up
    followup_question = generate_followup_question(learning_goal)
    print(f"\n🔍 Reflect on this:\n{followup_question}\n")

    # Time response (to evaluate latency later)
    start_time = time.time()
    user_response = input("✍️ Your response:\n> ")
    end_time = time.time()

    time_taken = round(end_time - start_time, 2)
    timestamp = datetime.now().isoformat()

    print("\n🕒 Time to respond:", time_taken, "seconds")
    print("🗓️ Timestamp:", timestamp)

    return {
        "learning_goal": learning_goal,
        "followup_question": followup_question,
        "response": user_response,
        "time_taken": time_taken,
        "timestamp": timestamp
    }

# Run prototype
if __name__ == "__main__":
    result = agent_one()
    print("\n✅ DATA CAPTURED:")
    for k, v in result.items():
        print(f"{k}: {v}")

def analyze_response_gemma(response_text, time_taken, session_num, goal, history=None):
    prompt = f"""
You are an expert language analyst. A learner wants to improve at: "{goal}".

Here is their reflective response:
"{response_text}"

Evaluate on a 0–1 scale:
1. Conceptual Vocabulary (depth, variety, originality)
2. Clarity of Expression
3. Emotional Tone Strength

Respond in JSON like this:
{{
  "conceptual_vocab_score": 0.0,
  "clarity_score": 0.0,
  "emotional_tone_score": 0.0
}}
"""

    try:
        result = client.chat.completions.create(
            model="google/gemma-3n-e2b-it:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=200
        )
        content = result.choices[0].message.content.strip()
        print("🔍 GEMMA RAW OUTPUT:", content)

        scores = json.loads(content)

    except Exception as e:
        print("⚠️ Gemma scoring failed:", e)
        scores = {
            "conceptual_vocab_score": 0.5,
            "clarity_score": 0.5,
            "emotional_tone_score": 0.5
        }

    # Optional: latency variability
    if history and len(history) > 1:
        previous_times = [s["time_taken"] for s in history]
        latency_std = round(statistics.stdev(previous_times), 2)
    else:
        latency_std = None

    return {
        "conceptual_vocab_score": scores["conceptual_vocab_score"],
        "clarity_score": scores["clarity_score"],
        "emotional_tone_score": scores["emotional_tone_score"],
        "time_to_completion": time_taken,
        "latency_variability": latency_std
    }



In [ ]:
from openai import OpenAI
import time
from datetime import datetime
import json
import statistics

# Setup OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

def generate_followup_question(learning_goal):
    prompt = f"""You are a motivational learning coach. A learner just said they want to learn: "{learning_goal}".

Generate one open-ended, thoughtful question that will help them expand deeply on their personal motivation, challenges, or relationship to this topic. Be emotionally engaging and not too abstract."""

    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1-0528-qwen3-8b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300
        )
        print("✅ DeepSeek response received.")
        print("🔧 FULL RESPONSE:", response)

        content = response.choices[0].message.content.strip() if response.choices else ""
        if not content:
            print("⚠️ Empty content. Using fallback.")
            content = "What does writing mean to you personally, and how do you want it to impact your future?"

        return content

    except Exception as e:
        print(f"⚠️ DeepSeek generation failed: {e}")
        return "What inspires your desire to write, and what do you hope to express through it?"

def agent_one():
    print("🎯 Welcome to the Adaptive Learning System (Prototype Agent 1)")
    learning_goal = input("🧠 What are you trying to learn today?\n> ")

    # Generate personalized follow-up
    followup_question = generate_followup_question(learning_goal)
    print(f"\n🔍 Reflect on this:\n{followup_question}\n")

    # Time response
    start_time = time.time()
    user_response = input("✍️ Your response:\n> ")
    end_time = time.time()

    time_taken = round(end_time - start_time, 2)
    timestamp = datetime.now().isoformat()

    print("\n🕒 Time to respond:", time_taken, "seconds")
    print("🗓️ Timestamp:", timestamp)

    return {
        "learning_goal": learning_goal,
        "followup_question": followup_question,
        "response": user_response,
        "time_taken": time_taken,
        "timestamp": timestamp
    }

def analyze_response_gemma(response_text, time_taken, session_num, goal, history=None):
    prompt = f"""
You are an expert language analyst. A learner wants to improve at: "{goal}".

Here is their reflective response:
"{response_text}"

Evaluate on a 0–1 scale:
1. Conceptual Vocabulary (depth, variety, originality)
2. Clarity of Expression
3. Emotional Tone Strength

Respond in JSON like this:
{{
  "conceptual_vocab_score": 0.0,
  "clarity_score": 0.0,
  "emotional_tone_score": 0.0
}}
"""
    try:
        result = client.chat.completions.create(
            model="google/gemma-3n-e2b-it:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=200
        )
        print("🔍 FULL GEMMA RESPONSE OBJECT:", result)
        content = result.choices[0].message.content.strip()
        print("🔍 GEMMA OUTPUT TEXT:", content)
        scores = json.loads(content)
    except Exception as e:
        print("⚠️ Gemma scoring failed:", e)
        scores = {
            "conceptual_vocab_score": 0.5,
            "clarity_score": 0.5,
            "emotional_tone_score": 0.5
    }

    # Compute latency variability if history exists
    if history and len(history) > 1:
        previous_times = [s["time_taken"] for s in history]
        latency_std = round(statistics.stdev(previous_times), 2)
    else:
        latency_std = None

    return {
        "conceptual_vocab_score": scores["conceptual_vocab_score"],
        "clarity_score": scores["clarity_score"],
        "emotional_tone_score": scores["emotional_tone_score"],
        "time_to_completion": time_taken,
        "latency_variability": latency_std
    }

# Main runner
if __name__ == "__main__":
    history = []

    result = agent_one()
    analysis = analyze_response_gemma(
        response_text=result["response"],
        time_taken=result["time_taken"],
        session_num=1,
        goal=result["learning_goal"],
        history=history
    )

    print("\n✅ DATA CAPTURED:")
    for k, v in result.items():
        print(f"{k}: {v}")

    print("\n📊 ANALYSIS SCORES:")
    for k, v in analysis.items():
        print(f"{k}: {v}")

    # Save to history (for future sessions)
    history.append(result)


# this works partly for what we want done.

In [ ]:
# this version of the code works really well, and is providing us with a decent feedback that should help with what we are trying ti figure. 
# -------- this version works. ----------------

from openai import OpenAI
import time
from datetime import datetime
import json
import statistics
import re

# Setup OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

def generate_followup_question(learning_goal):
    prompt = f"""You are a motivational learning coach. A learner just said they want to learn: "{learning_goal}".

Generate one open-ended, thoughtful question that will help them expand deeply on their personal motivation, challenges, or relationship to this topic. Be emotionally engaging and not too abstract."""

    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1-0528-qwen3-8b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300
        )
        print("✅ DeepSeek response received.")
        print("🔧 FULL RESPONSE:", response)

        content = response.choices[0].message.content.strip() if response.choices else ""
        if not content:
            print("⚠️ Empty content. Using fallback.")
            content = "What does writing mean to you personally, and how do you want it to impact your future?"

        return content

    except Exception as e:
        print(f"⚠️ DeepSeek generation failed: {e}")
        return "What inspires your desire to write, and what do you hope to express through it?"

def agent_one():
    print("🎯 Welcome to the Adaptive Learning System (Prototype Agent 1)")
    learning_goal = input("🧠 What are you trying to learn today?\n> ")

    # Generate personalized follow-up
    followup_question = generate_followup_question(learning_goal)
    print(f"\n🔍 Reflect on this:\n{followup_question}\n")

    # Time response
    start_time = time.time()
    user_response = input("✍️ Your response:\n> ")
    end_time = time.time()

    time_taken = round(end_time - start_time, 2)
    timestamp = datetime.now().isoformat()

    print("\n🕒 Time to respond:", time_taken, "seconds")
    print("🗓️ Timestamp:", timestamp)

    return {
        "learning_goal": learning_goal,
        "followup_question": followup_question,
        "response": user_response,
        "time_taken": time_taken,
        "timestamp": timestamp
    }

def analyze_response_gemma(response_text, time_taken, session_num, goal, history=None):
    prompt = f"""
You are an expert language analyst. A learner wants to improve at: "{goal}".

Here is their reflective response:
"{response_text}"

Evaluate on a 0–1 scale:
1. Conceptual Vocabulary (depth, variety, originality)
2. Clarity of Expression
3. Emotional Tone Strength

Respond in JSON like this:
{{
  "conceptual_vocab_score": 0.0,
  "clarity_score": 0.0,
  "emotional_tone_score": 0.0
}}

Then write 1 sentence each explaining why the learner scored that way:
- Conceptual Vocabulary: ...
- Clarity: ...
- Emotional Tone: ...
"""
    try:
        result = client.chat.completions.create(
            model="google/gemma-3n-e2b-it:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=300
        )
        raw_output = result.choices[0].message.content.strip()
        print("🔍 GEMMA RAW OUTPUT:", raw_output)

        # Extract JSON
        json_block = re.search(r"\{.*?\}", raw_output, re.DOTALL)
        if json_block:
            scores = json.loads(json_block.group(0))
        else:
            raise ValueError("Could not parse JSON block")

        # Extract explanation lines
        explanations = {
            "conceptual_vocab_feedback": "Not found",
            "clarity_feedback": "Not found",
            "emotional_tone_feedback": "Not found"
        }
        lines = raw_output.splitlines()
        for line in lines:
            if line.lower().startswith("- conceptual"):
                explanations["conceptual_vocab_feedback"] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("- clarity"):
                explanations["clarity_feedback"] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("- emotional"):
                explanations["emotional_tone_feedback"] = line.split(":", 1)[1].strip()

    except Exception as e:
        print("⚠️ Gemma scoring failed:", e)
        scores = {
            "conceptual_vocab_score": 0.5,
            "clarity_score": 0.5,
            "emotional_tone_score": 0.5
        }
        explanations = {
            "conceptual_vocab_feedback": "Default due to error.",
            "clarity_feedback": "Default due to error.",
            "emotional_tone_feedback": "Default due to error."
        }

    # Latency variability
    if history and len(history) > 1:
        previous_times = [s["time_taken"] for s in history]
        latency_std = round(statistics.stdev(previous_times), 2)
    else:
        latency_std = None

    return {
        **scores,
        **explanations,
        "time_to_completion": time_taken,
        "latency_variability": latency_std
    }

# Main runner
if __name__ == "__main__":
    history = []

    result = agent_one()
    analysis = analyze_response_gemma(
        response_text=result["response"],
        time_taken=result["time_taken"],
        session_num=1,
        goal=result["learning_goal"],
        history=history
    )

    print("\n✅ DATA CAPTURED:")
    for k, v in result.items():
        print(f"{k}: {v}")

    print("\n📊 ANALYSIS SCORES:")
    for k in ["conceptual_vocab_score", "clarity_score", "emotional_tone_score"]:
        print(f"{k}: {analysis[k]}")

    print("\n🧠 FEEDBACK:")
    for k in ["conceptual_vocab_feedback", "clarity_feedback", "emotional_tone_feedback"]:
        label = k.replace("_feedback", "").replace("_", " ").capitalize()
        print(f"- {label}: {analysis[k]}")

    history.append(result)


✅ DeepSeek response received.
🔧 FULL RESPONSE: ChatCompletion(id='gen-1752458931-K8twJo27lnp4ZE5HPDhy', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="Okay, I understand you're", refusal=None, role='assistant', function_call=None, tool_calls=None, reasoning="Okay, the user wants me to act as a motivational learning coach. Their query is a bit vague because they haven't specified what topic they want to learn. But they're asking me to create an open-ended, thoughtful question that will help them expand on their personal motivation, challenges, or relationship to this topic.\n\nI need to craft a question that's emotionally engaging and not too abstract. This means focusing on their personal experience and feelings about learning. Since they didn't specify the topic, I'll need to keep my response general enough to work for whatever topic they might be interested in.\n\nThe question should help them reflect deeply on why they want to 